# Interactive Value at Risk (VaR) Calculator

**Purpose:** This notebook provides a hands-on, interactive way to understand and calculate Value at Risk (VaR), a key measure of market risk. It explores different VaR calculation methodologies (Historical Simulation and Parametric) and allows you to see how various inputs affect VaR outcomes.

**Target Audience:** Students of finance, risk management professionals, CFA candidates, or anyone interested in a practical introduction to VaR.

**How to Use:** Run the cells sequentially. Use the interactive sliders and input fields that appear to change parameters and see how the calculations and visualizations update in real-time.

## Section 1: Setup and Data Generation

First, we import the necessary Python libraries and set up some functions to generate sample portfolio return data. This ensures the notebook is self-contained and can be run without needing to fetch live data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from ipywidgets import interact, fixed, FloatSlider, IntSlider

sns.set_style('whitegrid')

# --- Data Generation ---
def generate_sample_returns(n_days=1000, mean_return=0.0005, std_dev=0.015):
    """Generates a sample series of normally distributed daily returns."""
    np.random.seed(42) # for reproducibility
    returns = np.random.normal(mean_return, std_dev, n_days)
    return pd.Series(returns, name='Daily Returns')

# Generate our base dataset
initial_portfolio_value = 1000000
returns_data = generate_sample_returns()

print(f"Generated {len(returns_data)} days of sample return data.")
print("Sample Mean Daily Return:", returns_data.mean())
print("Sample Std Dev of Daily Return:", returns_data.std())

## Section 2: Historical Simulation VaR

The Historical Simulation method is the most straightforward way to calculate VaR. It makes no assumptions about the distribution of returns and simply uses the actual historical returns to model what could happen in the future.

**Method:**
1. Take a series of historical returns.
2. Sort the returns from worst to best.
3. Find the return at the desired percentile. For example, for a 99% VaR, we find the 1st percentile's worst loss.

In [ ]:
def calculate_historical_var(returns, confidence_level=99, portfolio_value=initial_portfolio_value):
    """Calculates VaR using the historical simulation method."""
    # For a 99% confidence level, we want the 1st percentile (100 - 99 = 1)
    var_percentile = 100 - confidence_level
    var_return = np.percentile(returns, var_percentile)
    var_amount = var_return * portfolio_value
    return var_return, var_amount

def plot_historical_var(returns, confidence_level, portfolio_value):
    var_return, var_amount = calculate_historical_var(returns, confidence_level, portfolio_value)
    
    plt.figure(figsize=(12, 6))
    sns.histplot(returns, bins=50, kde=True, color='skyblue', stat='density')
    plt.axvline(x=var_return, color='red', linestyle='--', linewidth=2, label=f'{confidence_level}% VaR Threshold')
    plt.title(f'Historical Daily Returns Distribution with {confidence_level}% VaR')
    plt.xlabel('Daily Return')
    plt.ylabel('Density')
    plt.legend()
    plt.show()
    
    print(f"Based on historical simulation:")
    print(f"The return at the {100-confidence_level:.1f}th percentile is {var_return:.4f}.")
    print(f"For a portfolio of ${portfolio_value:,.0f}, the {confidence_level}% VaR is ${-var_amount:,.2f}.")
    print(f"This means there is a {100-confidence_level:.1f}% chance of losing more than ${-var_amount:,.2f} in one day.")

# Interactive widget for Historical VaR
interact(
    plot_historical_var,
    returns=fixed(returns_data),
    confidence_level=FloatSlider(min=90.0, max=99.9, step=0.1, value=99.0, description='Confidence Level (%)'),
    portfolio_value=fixed(initial_portfolio_value)
);

## Section 3: Parametric (Variance-Covariance) VaR

The Parametric method assumes that returns follow a specific distribution, typically the normal distribution. It uses the mean and standard deviation of historical returns to calculate VaR.

**Method:**
1. Calculate the mean (μ) and standard deviation (σ) of historical returns.
2. Find the z-score corresponding to the desired confidence level from the standard normal distribution.
3. Apply the formula: `VaR = Portfolio Value * (μ + z * σ)`

In [ ]:
def calculate_parametric_var(returns, confidence_level=99, portfolio_value=initial_portfolio_value):
    """Calculates VaR using the parametric (variance-covariance) method."""
    mean = returns.mean()
    std_dev = returns.std()
    
    # For a 99% confidence level, we want the z-score for the 1st percentile
    alpha = 1 - (confidence_level / 100)
    z_score = norm.ppf(alpha) # ppf is the Percent Point Function (inverse of cdf)
    
    var_return = mean + z_score * std_dev
    var_amount = var_return * portfolio_value
    return var_return, var_amount

def plot_parametric_var(returns, confidence_level, portfolio_value):
    mean = returns.mean()
    std_dev = returns.std()
    var_return, var_amount = calculate_parametric_var(returns, confidence_level, portfolio_value)
    
    plt.figure(figsize=(12, 6))
    # Plot the theoretical normal distribution
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    y = norm.pdf(x, mean, std_dev)
    plt.plot(x, y, label='Normal Distribution', color='blue')
    
    # Shade the tail
    plt.fill_between(x, y, where=(x < var_return), color='red', alpha=0.5, label=f'{100-confidence_level:.1f}% Tail')
    
    plt.axvline(x=var_return, color='red', linestyle='--', linewidth=2, label=f'{confidence_level}% VaR Threshold')
    plt.title(f'Parametric VaR based on Normal Distribution')
    plt.xlabel('Daily Return')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.show()
    
    print(f"Based on the parametric (variance-covariance) method:")
    print(f"The return at the {100-confidence_level:.1f}th percentile is {var_return:.4f}.")
    print(f"For a portfolio of ${portfolio_value:,.0f}, the {confidence_level}% VaR is ${-var_amount:,.2f}.")
    print(f"This assumes returns are normally distributed with a mean of {mean:.4f} and std dev of {std_dev:.4f}.")

# Interactive widget for Parametric VaR
interact(
    plot_parametric_var,
    returns=fixed(returns_data),
    confidence_level=FloatSlider(min=90.0, max=99.9, step=0.1, value=99.0, description='Confidence Level (%)'),
    portfolio_value=fixed(initial_portfolio_value)
);

## Section 4: Comparing Methods and Limitations

*   **Historical VaR:** 
    *   **Pros:** Easy to understand, doesn't assume a normal distribution (captures fat tails if they existed in the past).
    *   **Cons:** Assumes the past is a good representation of the future; sensitive to the lookback period.
*   **Parametric VaR:** 
    *   **Pros:** Easy to calculate, only requires mean and standard deviation.
    *   **Cons:** Relies heavily on the assumption of normality, which is often not true for financial returns (they tend to have 'fat tails'). Can significantly underestimate risk if the distribution is not normal.

Notice how the VaR numbers from the two methods can differ slightly. The choice of method depends on the user's assumptions about return distributions and the available data.